In [2]:
import keras as keras
import tensorflow as tf
from keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from keras.layers import Bidirectional
from keras.utils import pad_sequences
from keras.callbacks import EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from keras import regularizers
from keras.layers import Input, LSTM, Dense, Concatenate
from keras.models import Model
from itertools import permutations
import keras.backend as K
import pandas as pd
import numpy as np

2023-12-10 15:57:55.484080: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-10 15:57:55.484170: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-10 15:57:55.532042: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-10 15:57:55.634206: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-10 15:57:56.914577: W tensorflow/compiler/tf2

In [3]:
def permutation_invariant_loss(y_true, y_pred):
    # Sort the true and predicted values along the last axis
    y_true_sorted = tf.sort(y_true, axis=-1)
    y_pred_sorted = tf.sort(y_pred, axis=-1)

    # Calculate the mean absolute error between the sorted true and predicted values
    return K.mean(K.abs(y_true_sorted - y_pred_sorted), axis=-1)


In [4]:
csv_file_path = 'sinusoid_dataset.csv'
df = pd.read_csv(csv_file_path)
features = df['Feature'].apply(lambda x: np.array([float(val.strip("[]")) for val in x.split()])).values
labels = df['Label'].apply(lambda x: np.array([float(val.strip("[]")) for val in x.split(',')])).values
print(features.shape)
print(labels.shape)

(200000,)
(200000,)


In [5]:
padded_array = pad_sequences(labels, padding='post', maxlen=3)
print(padded_array.shape)
labels = np.vstack(padded_array)
features = np.vstack(features)
#scaler = MinMaxScaler()
#features = scaler.fit_transform(features)

(200000, 3)


In [6]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

X_train_tensor = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train_tensor = tf.convert_to_tensor(y_train, dtype=tf.float32)
X_test_tensor = tf.convert_to_tensor(X_test, dtype=tf.float32)
y_test_tensor = tf.convert_to_tensor(y_test, dtype=tf.float32)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train_categorical shape:", y_train.shape)
print("y_test_categorical shape:", y_test.shape)

2023-12-10 15:58:45.311116: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:08:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-10 15:58:45.472754: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:08:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-10 15:58:45.472831: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:08:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-10 15:58:45.476825: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:08:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-10 15:58:45.476907: I external/local_xla/xla/stream_executor

X_train shape: (160000, 299)
X_test shape: (40000, 299)
y_train_categorical shape: (160000, 3)
y_test_categorical shape: (40000, 3)


In [7]:
# Define three LSTM branches
inputs = keras.Input(shape=(299,1))
# Branch 1
lstm_branch_1 = LSTM(units=128, return_sequences=True)(inputs)
flatten_1 = layers.Flatten()(lstm_branch_1)
dense_1 = Dense(units=32, activation='tanh')(flatten_1)
output_branch_1 = Dense(units=1, activation='linear', name='output_branch_1')(dense_1)

# Branch 2
lstm_branch_2 = LSTM(units=128, return_sequences=True)(lstm_branch_1)  # Pass the output of branch 1
flatten_2 = layers.Flatten()(lstm_branch_2)
dense_2 = Dense(units=32, activation='tanh')(flatten_2)
output_branch_2 = Dense(units=1, activation='linear', name='output_branch_2')(dense_2)

# Branch 3
lstm_branch_3 = LSTM(units=128, return_sequences=True)(lstm_branch_2)  # Pass the output of branch 2
flatten_3 = layers.Flatten()(lstm_branch_3)
dense_3 = Dense(units=32, activation='tanh')(flatten_3)
output_branch_3 = Dense(units=1, activation='linear', name='output_branch_3')(dense_3)

# Concatenate the outputs of the three branches
merged_output = Concatenate(axis=-1)([output_branch_1, output_branch_2, output_branch_3])

# Define the model
model = Model(inputs=inputs, outputs=merged_output)
print(model.summary())

2023-12-10 15:58:49.776524: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 299, 1)]             0         []                            
                                                                                                  
 lstm (LSTM)                 (None, 299, 128)             66560     ['input_1[0][0]']             
                                                                                                  
 lstm_1 (LSTM)               (None, 299, 128)             131584    ['lstm[0][0]']                
                                                                                                  
 lstm_2 (LSTM)               (None, 299, 128)             131584    ['lstm_1[0][0]']              
                                                                                              

In [9]:
model.compile(optimizer='adam',
              loss=permutation_invariant_loss)

model.fit(X_train_tensor, y_train_tensor, epochs=20, batch_size=256, validation_split=0.2)

test_loss = model.evaluate(X_test_tensor, y_test_tensor)
print(f'Test Loss: {test_loss:.4f}')

Epoch 1/20
500/500 [==============================] - 68s 127ms/step - loss: 22.9920 - val_loss: 19.7512
Epoch 2/20
500/500 [==============================] - 62s 124ms/step - loss: 16.9730 - val_loss: 14.5358
Epoch 3/20
500/500 [==============================] - 66s 131ms/step - loss: 12.4282 - val_loss: 10.5786
Epoch 4/20
500/500 [==============================] - 63s 125ms/step - loss: 9.1415 - val_loss: 8.0346
Epoch 5/20
500/500 [==============================] - 60s 121ms/step - loss: 6.8589 - val_loss: 5.9567
Epoch 6/20
500/500 [==============================] - 68s 135ms/step - loss: 5.3610 - val_loss: 4.8201
Epoch 7/20
500/500 [==============================] - 61s 123ms/step - loss: 4.6130 - val_loss: 4.3508
Epoch 8/20
500/500 [==============================] - 64s 129ms/step - loss: 4.1098 - val_loss: 3.9080
Epoch 9/20
500/500 [==============================] - 64s 127ms/step - loss: 3.8307 - val_loss: 3.7800
Epoch 10/20
500/500 [==============================] - 205s 410ms/s

TypeError: cannot unpack non-iterable float object

In [12]:
test_loss = model.evaluate(X_test_tensor, y_test_tensor)
print(f'Test Loss: {test_loss:.4f}')

1250/1250 [==============================] - 49s 39ms/step - loss: 3.8659
Test Loss: 3.8659


In [11]:
csv_file_path = 'samplesreal.csv'  # Replace with the actual path to your CSV file
df = pd.read_csv(csv_file_path)

# Extract features and labels
features = df['Feature'].apply(lambda x: np.array([float(val.strip("[]")) for val in x.split(',')])).values
labels = [[label] for label in df['Label'].tolist()]
# Convert the string of up to 5 numbers to a NumPy array of floats

padded_array = pad_sequences(labels, padding='post', maxlen=3)
labels = np.vstack(padded_array)
features = np.vstack(features)
means = np.mean(features, axis=1, keepdims=True)
features = features - means
features = features * 50

print("Real Sinusoid Tests:\n")
for i in range(8):
  print(labels[i:i+1])
  print(model.predict(features[i:i+1]))

print("Synthetic Sinusoid Tests:\n")
for i in range(100):
  print(y_test[i:i+1])
  print(model.predict(X_test[i:i+1]))

Real Sinusoid Tests:

[[1 0 0]]
1/1 [==============================] - 0s 37ms/step
[[1.4368706e+00 1.3972968e+01 9.7100623e-04]]
[[10  0  0]]
1/1 [==============================] - 0s 36ms/step
[[7.2691379e+00 5.3803921e-03 9.7100623e-04]]
[[19  0  0]]
1/1 [==============================] - 0s 60ms/step
[[1.8278706e+01 5.3803921e-03 9.7100623e-04]]
[[27  0  0]]
1/1 [==============================] - 0s 32ms/step
[[2.8930485e+01 5.3803921e-03 9.7100623e-04]]
[[36  0  0]]
1/1 [==============================] - 0s 31ms/step
[[3.4932632e+01 5.3803921e-03 9.7100623e-04]]
[[45  0  0]]
1/1 [==============================] - 0s 34ms/step
[[4.7110653e+01 5.3803921e-03 9.7100623e-04]]
[[54  0  0]]
1/1 [==============================] - 0s 35ms/step
[[6.6081284e+01 4.6685005e+01 9.7100623e-04]]
[[90  0  0]]
1/1 [==============================] - 0s 57ms/step
[[9.1053139e+01 5.3803921e-03 9.7100623e-04]]
Synthetic Sinusoid Tests:

[[69  0  0]]
1/1 [==============================] - 0s 84ms/step
[